Este notebook tiene la finalidad de documentar la última limpieza de nuestro conjunto de datos y la generación de esta misma. En resumen, aquí se desgloza un arduo proceso de codificación, eliminación de registros espurios, corección de variables anteriores y establecer funciones relevantes para el modelado.

Recordamos que nuestro equipo está conformado por

+ Wendy Areli Ramírez Burgos
+ Gustavo Corona Ojeda
+ Carlos Fernando Vásquez Guerra

Cada uno de nosotros nos dimos a la tarea de analizar las ofertas laborales más a fondo para encontrar posibles registros engañosos en cuanto a su contenido y también para obtener variables útiles que nos proporcionarán información relevante para la modelación.

Así que este notebook solo debe ser considerado como un anexo a nuestro proyecto. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
warnings.filterwarnings("ignore")

jobs = pd.read_csv("DataScience_jobs.csv")

Algunos esquemas salariales se corrigieron, por lo que debemos ajustar las variables relacionadas al salario

In [2]:
def suma_WNans(v_min, v_max):
    if np.isnan(v_min):
        return v_max
    if np.isnan(v_max):
        return v_min
    else:
        return (v_min + v_max)/2
    
def original_salary(money, currency, scheme):
    #Convertimos a dolares (USD) para estandarizar nuestros valores
    if currency == "CAD":
        money = money/0.75
    if currency == "MXN":
        money = money/0.050
    if scheme == "hour":
        money = money/8/5/4
    if scheme == "year":
        money = money*12
    if scheme == "week":
        money = money/4
    return(money)

#jobs["Salario promedio"] = jobs.apply(lambda x: suma_WNans(x["Salario mínimo"], x["Salario máximo"]), axis=1)
    
def standar_salary_individual(v_min, v_max, currency, scheme):
    money = suma_WNans(v_min, v_max)#Para sacar el salario promedio
    #Convertimos a dolares (USD) para estandarizar nuestros valores
    if currency == "CAD":
        money = money*0.75
    if currency == "MXN":
        money = money*0.050
    #Pasamos todos los salarios a pagos mensuales (vamos a suponer una jornada legal de 8 horas)
    if scheme == "hour":
        money = money*8*5*4 #8 horas por 5 días por 4 semanas
    if scheme == "year":
        money = money/12
    if scheme == "week":
        money = money*4
    return(money)

In [3]:
index_to_fix_salary_carlos = list(map(lambda x: x-1, [246, 249, 253, 255, 263, 265, 320, 321, 351, 384, 385, 465]))
scheme_to_fix_salary_carlos = ["month", "month", "month", "week", "month", "month", "month", "month", "month", "month", "month", "month"]

In [4]:
jobs.iloc[index_to_fix_salary_carlos]

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,Modalidad,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio
245,Analista de Bases de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-15,FULL_TIME,MX,not specified,2,Not specified,1,750.0000
248,Analista de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-15,FULL_TIME,MX,not specified,2,Not specified,1,750.0000
252,Analista de Datos Junior,SOC Asesores,not specified,not specified,522.3500,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-08-24,2022-10-15,FULL_TIME,MX,not specified,4,Junior,1,522.3500
254,Analista de Datos,Grupo Ortiz SA DE CV,Michoacán,Morelia,400.0000,500.0,MXN,Forma Parte de Grupo Ortiz.Somos una de las em...,not specified,2022-09-14,2022-10-15,FULL_TIME,MX,month,4,Not specified,1,450.0000
262,Analista de Información,Servicios de Telecomunicaciones,not specified,not specified,551.7000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-14,2022-10-15,FULL_TIME,MX,not specified,3,Not specified,1,551.7000
264,Analista de Bases de Datos,GRUPO CONSULTORES,not specified,not specified,500.0000,600.0,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-08,2022-10-15,FULL_TIME,MX,not specified,3,Not specified,1,550.0000
319,Analista de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-14,FULL_TIME,MX,not specified,2,Not specified,1,750.0000
320,Analista de Bases de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-14,FULL_TIME,MX,not specified,2,Not specified,1,750.0000
350,IT QA Analyst Lead (Remote),Integon Service Co.,not specified,not specified,2128.5335,NaN,MXN,Primary Purpose: This position is a senior mem...,not specified,2022-09-03,2022-10-16,FULL_TIME,MX,not specified,5,Not specified,1,2128.5335
383,Analista de Información,Keyser,not specified,not specified,750.0000,750.0,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-08-16,2022-10-16,FULL_TIME,MX,not specified,5,Not specified,0,750.0000


In [5]:
for i in range(len(index_to_fix_salary_carlos)):
    index = index_to_fix_salary_carlos[i]
    jobs.at[index, "Esquema de pago"] = scheme_to_fix_salary_carlos[i]
    jobs.at[index, 'Salario promedio'] = standar_salary_individual(original_salary(jobs.iloc[index]["Salario mínimo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]), 
                                                                     original_salary(jobs.iloc[index]["Salario máximo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]),
                                                                     jobs.iloc[index]["Tipo de moneda"], 
                                                                     jobs.iloc[index]["Esquema de pago"]
                                                                    )

In [6]:
jobs.iloc[index_to_fix_salary_carlos]

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,Modalidad,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio
245,Analista de Bases de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-15,FULL_TIME,MX,month,2,Not specified,1,750.0000
248,Analista de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-15,FULL_TIME,MX,month,2,Not specified,1,750.0000
252,Analista de Datos Junior,SOC Asesores,not specified,not specified,522.3500,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-08-24,2022-10-15,FULL_TIME,MX,month,4,Junior,1,522.3500
254,Analista de Datos,Grupo Ortiz SA DE CV,Michoacán,Morelia,400.0000,500.0,MXN,Forma Parte de Grupo Ortiz.Somos una de las em...,not specified,2022-09-14,2022-10-15,FULL_TIME,MX,week,4,Not specified,1,450.0000
262,Analista de Información,Servicios de Telecomunicaciones,not specified,not specified,551.7000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-14,2022-10-15,FULL_TIME,MX,month,3,Not specified,1,551.7000
264,Analista de Bases de Datos,GRUPO CONSULTORES,not specified,not specified,500.0000,600.0,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-08,2022-10-15,FULL_TIME,MX,month,3,Not specified,1,550.0000
319,Analista de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-14,FULL_TIME,MX,month,2,Not specified,1,750.0000
320,Analista de Bases de Datos,TRES R,not specified,not specified,750.0000,NaN,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-09-05,2022-10-14,FULL_TIME,MX,month,2,Not specified,1,750.0000
350,IT QA Analyst Lead (Remote),Integon Service Co.,not specified,not specified,2128.5335,NaN,MXN,Primary Purpose: This position is a senior mem...,not specified,2022-09-03,2022-10-16,FULL_TIME,MX,month,5,Not specified,1,2128.5335
383,Analista de Información,Keyser,not specified,not specified,750.0000,750.0,MXN,Esta vacante viene de la bolsa de empleo Talen...,not specified,2022-08-16,2022-10-16,FULL_TIME,MX,month,5,Not specified,0,750.0000


Ahora lo hacemos con las correcciones de Gus y Wen. Como no sabemos si se hicieron correcciones o no al salario, supondremos que el esquema salarial en sus data sets son los correctos. Así que vamos a regresar el salario a los datos originales, vamos a sustituír los esquemas malos por los buenos y luego volver a normalizar la variable

In [7]:
jobs_wen_squeme = pd.read_csv("LastClean/DataScience_jobs_wen_scheme.csv")
jobs_gus_squeme = pd.read_csv("LastClean/DataScience_jobs_gus_scheme.csv")
jobs_gus_squeme.drop([235, 236], inplace=True)

In [8]:
index_to_fix_salary_wen = list(jobs_wen_squeme["Index"]-1)
for i in range(len(index_to_fix_salary_wen)):
    index = index_to_fix_salary_wen[i]
    jobs.at[index, "Esquema de pago"] = jobs_wen_squeme["Esquema de pago"][i]
    jobs.at[index, 'Salario promedio'] = standar_salary_individual(original_salary(jobs.iloc[index]["Salario mínimo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]), 
                                                                     original_salary(jobs.iloc[index]["Salario máximo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]),
                                                                     jobs.iloc[index]["Tipo de moneda"], 
                                                                     jobs.iloc[index]["Esquema de pago"]
                                                                    )

In [9]:
index_to_fix_salary_gus = list(jobs_gus_squeme["Unnamed: 0"]-1)
for i in range(len(index_to_fix_salary_gus)):
    index = int(index_to_fix_salary_gus[i])
    jobs.at[index, "Esquema de pago"] = jobs_gus_squeme["Esquema de pago"][i]
    jobs.at[index, 'Salario promedio'] = standar_salary_individual(original_salary(jobs.iloc[index]["Salario mínimo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]), 
                                                                     original_salary(jobs.iloc[index]["Salario máximo"], jobs.iloc[index]["Tipo de moneda"], jobs.iloc[index]["Esquema de pago"]),
                                                                     jobs.iloc[index]["Tipo de moneda"], 
                                                                     jobs.iloc[index]["Esquema de pago"]
                                                                    )

Ahora, eliminamos las observaciones que detectamos que no están relacionadas a los trabajos de un cientifico de datos

In [10]:
carlos_drop = list(map(lambda x: x-1, [234, 240, 241, 243, 261, 262, 267, 271, 272, 274, 275, 276, 285, 286, 288, 290, 291, 292, 293, 294, 296, 298, 299, 301, 303, 305, 306, 308, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 325, 327, 328, 329, 330, 333, 334, 335, 336, 337, 339, 341, 342, 345, 347, 348, 349, 356, 357, 363, 364, 365, 366, 367, 368, 369, 371, 373, 374, 375, 378, 382, 387, 388, 389, 390, 391, 392, 394, 396, 397, 398, 399, 404, 406, 407, 409, 410, 412, 413, 418, 420, 421, 422, 423, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 439, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 453, 454, 459, 461, 464, 467]))
gus_drop = list(map(lambda x: x-1, [471, 479, 482, 485, 492, 494, 500, 501, 502, 503, 504, 506, 507, 509, 512, 513, 514, 515, 518, 519, 522, 537, 545, 547, 554, 564, 603, 604, 606, 639, 641, 643, 644, 645, 646, 647, 648, 649, 650, 654, 655, 661, 672, 673, 675, 697]))
wen_drop = list(map(lambda x: x-1, [65, 75, 78, 83, 84, 90, 91, 95, 100, 101, 112, 113, 115, 116, 119, 129, 133, 136, 140, 142]))

In [11]:
jobs.drop(carlos_drop + gus_drop + wen_drop, inplace=True)
jobs.reset_index(drop=True, inplace=True)

Ahora, agregamos las nuevas columnas

In [12]:
jobs_wen_NV = pd.read_csv("LastClean/DataScience_jobs_wen_NewVariables.csv")
jobs_fer_NV = pd.read_csv("LastClean/DataScience_jobs_fer_NewVariables.csv")
jobs_gus_NV = pd.read_csv("LastClean/DataScience_jobs_gus_NewVariables_V2.csv")
jobs_gus_NV.drop([189, 190], inplace=True)
jobs_gus_NV.rename(columns={"Experiencia(años)": "Experiencia (Años) "}, inplace=True)

In [13]:
NVariables = ['Lenguajes de programación', 'Modalidad.1', 'Estudios previos',
       'Experiencia (Años) ', 'Habilidades blandas', 'Idiomas']
for i in NVariables:
    jobs[i] = pd.concat([jobs_wen_NV[i], jobs_fer_NV[i], jobs_gus_NV[i]], ignore_index=True)
jobs.drop(324, inplace=True)
jobs.reset_index(inplace=True, drop=True)

In [14]:
jobs

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,Modalidad,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio,Lenguajes de programación,Modalidad.1,Estudios previos,Experiencia (Años),Habilidades blandas,Idiomas
0,Data Engineer with Azure Synapse,4urHires INC,not specified,not specified,8800.000000,9600.000000,USD,Azure EngineerStrong experience in SynapseJob ...,not specified,2022-09-16,2022-10-16,OTHER,EUA,hour,2,Not specified,0,9200.000000,"Azure, synapse,","Remote,","Azure engeneer,","Data engineer: 10 years, azure synapse: 5 years",-,NaN
1,Data Engineer,MARS Group,not specified,not specified,6348.800000,9600.000000,USD,Description: Data is a critical driver of this...,not specified,2022-09-14,2022-10-16,FULL_TIME,EUA,hour,8,Not specified,1,7974.400000,"Java, Scala, AWS, Docker, Kubernetes,","Remote,","B.S. degree in Computer Science or related field,","2+ years of experience in data engineering, j...","Ability to adapt and learn new technologies, ...",NaN
2,Data Engineer - ENTRY LEVEL,PCS Global Tech,California,Los Angeles,5000.000000,5833.333333,USD,Responsibilities· Analyze and organize raw dat...,Tecnologías de la información,2022-09-15,2022-10-16,FULL_TIME,EUA,year,5,Junior,1,5416.666667,"Java, Python, SQL,","Multiple locations,","Degree in Computer Science, IT, or similar fie...","Trainee,","Collaborate,",NaN
3,Entry-Level Azure Data Engineer,"Antra, Inc",not specified,not specified,5166.666667,5833.333333,USD,"About usAntra, Inc is a leading IT consultancy...",Tecnologías de la información,2022-09-13,2022-10-16,FULL_TIME,EUA,year,10,Junior,1,5500.000000,"SQL, Python, Scala, R, SQL Server, Oracle, MyS...","Remote,","Minimum Bachelor’s degree in Data Science, Bus...",".0-3 years,","Excellent problem-solving skills, ability to ...",NaN
4,Data Engineer only for OPT EAD,CloudBig Technology,not specified,not specified,8000.000000,9600.000000,USD,Work with Technical and business team to under...,not specified,2022-09-15,2022-10-16,FULL_TIME,EUA,hour,3,Not specified,1,8800.000000,"SQL, Data Warehouse,","Remote,",Master's,"Informatica: 1 year (Preferred), SQL: 1 year (...","work with other,",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,Sr. Business Systems Analyst,Infotek Consulting Services,Ontario,Toronto,6000.000000,8400.000000,CAD,"Need to know all: Confluence, Jira, Excel, Tab...",Tecnologías de la información,2022-09-16,2022-10-16,OTHER,CAN,hour,5,Senior,0,7200.000000,"SQL, Python, JSON, qtest, zephyr , HPALM / HPQC",-,-,5 years of experience working in a data or tec...,"Excellent communication, negotiation and stake...",-
508,Sr. Business Analyst,The Getch,Ontario,Toronto,9600.000000,10800.000000,CAD,The Opportunity: Business/Systems Infr/Cloud A...,Tecnologías de la información,2022-08-30,2022-10-16,FULL_TIME,CAN,hour,1,Senior,1,10200.000000,"AWS , BI",hybrid,B.S. degree in Finance or equivalent,"8 years with business planning, business model...",-,-
509,Sr. Business Analyst Capital Market,TechAheadCorp,Ontario,Toronto,6250.000000,NaN,CAD,- Minimum 8+ years of direct working experienc...,not specified,2022-09-09,2022-10-16,FULL_TIME,CAN,year,2,Senior,0,6250.000000,-,-,-,8+ years of direct working experience as a Bus...,-,-
510,Sr. Analyst BI,FinneyTaylor Consulting Group Ltd.,Alberta,Edmonton,4374.755000,5625.000000,CAD,Job Description : Our Client is looking for a ...,not specified,2022-09-02,2022-10-16,FULL_TIME,CAN,year,4,Senior,1,4999.877500,"SQL, SSAS, Dax, MS BI Stack, Power Query",-,"Bachelor’s degree in a related field (e.g., Co...",5+ years of relevant experience in Business In...,"can work independently, Strong oral and writte...",-


Con nuestra variable de lenguajes de programación, ya tenemos una mejor manera para calcular el número de lenguajes de programación. Pero primero necesitamos normalizar el texto y, por lo tanto, hacer trabajo de codificación

In [15]:
strange_character_1 = []
for i in (jobs["Lenguajes de programación"].str.contains("mathplotlib")):
    strange_character_1.append(i and not pd.isna(i))

In [16]:
for i in jobs["Lenguajes de programación"][strange_character_1]:
    print(i)

python, numpy, pandas, scikit-learn, tensorflow, pytorch, mathplotlib, opencv, scikit-image, keras, Dockers, Singularity, CNN, RNN, LSTM, GANs,


In [17]:
jobs["Lenguajes de programación"][strange_character_1]

46    python, numpy, pandas, scikit-learn, tensorflo...
Name: Lenguajes de programación, dtype: object

In [18]:
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(r"\:|\/|\)|\(|·|\*", ", ").str.lower()
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(r" or ", "").str.lower()
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(r";|\s|-", "").str.lower()
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(r",$", "")

In [19]:
chage_P = {"plot.ly":"plotly", "\.netcore1": "\.net", "acces": "access", "accesss": "access", "microsoftoffice": "microsoft office", "administrativa": "", "awsglue": "aws", "adobeacrobat": "adobe", "afni": "", "azureapimanagementtool": "azure","operl":"perl","tsql":"transact sql",
          "adobeanalitycs": "adobe", "advancedexcel": "excel", "agilemethodology": "agile", "agilescrumenvironments": "agile", "apachehadoop": "apache hadoop", "adobecreativesuite": "adobe", "awsclouddata": "aws", "azurestorage": "azure", "redhat": "red hat","tableu": "tableau",
          "agiletesting": "agile", "amazon":"aws", "amazonwebservices": "aws", "andbi": "", "anddbt": "", "anddeeplearningarchitectures": "", "apirest": "api", "apis": "api", "awsazure": "aws azure","bqquery": "bigquery", "cosmos db": "cosmos","solarisofrecemos": "solaris",
          "andhex.": "hex", "andsql": "sql", "andvertica": "vertica", "andwpf": "wpf", "angularjs": "angular", "+bigdata": "big data", "apachehbase": "apache hadoop", "andgooglecloudplatform": "google cloud platform", "cmms": "", "csv": "", "scikitlearn": "scikit learn",
          "awssueldo": "aws", "awstechstack.glue": "aws glue", "awswebservices": "aws", "azureanalysisservice": "azure", "azuredatabricks": "azure databricks", "azuredatafactory": "azure data factory", "cloudwatch": "aws cloud watch","datahubs": "datahub","spm": "",
          "azuredatalake": "azure data lake", "azuredevops": "azure devops", "azurefunctions": "azure", "basededatossql": "sql", "basesdedatosrelacionalesynorelacionales": "", "basesdedatosnorelacionales": "", "ants": "","costar": "","datapipelines": "","solrhadoop": "hadoop",
          "azuresqldb":"azure,sql", "azurestreamanalytics": "azure", "baesdedatos": "", "basesdedatos": "", "bashshellscripting": "bash", "bayesian": "", "bicomotableau": "tableau", "bi": "", "cot": "", "css.": "css", "css3": "css","css4": "css",",snowflake,": "snowflake cloud data",
          "blobstorage": "azure blob storage", "blockchain.": "", "c++sql": "c++,sql", "cassandrayhbase": "cassandra", "cnn": "", "cnns": "", "cognosanalytics": "", "coi": "", "com": "", "competenteenhojasdecálculo": "", "dash3": "dash", "serviciosweb":"","sparksql": "spark sql",
          "conocimientodeitilyscrumconocimiento": "scrum", "conocimientodemachinelearning": "", "core2asp.netcore": "", "cosmosdb": "cosmos db", "creacióndekpi’s": "", ".netcore1": ".net", "3gl": "", "adfv2": "azure data factory", "datamartsetlsql": "datamart","terradata": "teradata",
          "datascience":"","datascienceframeworks": "","datastage": "","datastudio": "data studio", "dataswarm": "","datawarehouse": "data warehouse", "dbt": "", "db2": "", "devops": "", "dockers": "docker", "dosbat": "", "googleanalytics": "google analytics", "statscan": "",
          "e.g.":"", "ec2": "aws ec2", "emr": "aws emr", "etl" : "", "exposuretomljob": "", "extremmeprogramming": "", "gcp": "google cloud platform", "geomesaapachenifi": "", "golang": "go", "googlesheets":"google sheets", "googlesuite":"google suite","soc": "","vistay7.office": "office",
          "googleworkspace":"google workspace", "hpalm": "hp alm", "hpqc": "hp qc", "javac++": "java,c++", "jsrelacionalesnorelacionales": "javascript", "jupyternotebooks": "jupyter notebooks", "keraspytorch": "keras,pytorch", "libreríasjquery": "jquery","sp.net": "",
          "linuxbasedservers": "linux server", "linuxservers": "linux server", "live engage": "", "managementstudio": "sql server management studio", "manejodebasededatos": "", "manejodeexcel": "excel", "mapinfoandarcgis": "mapinfo,arcgis","mapreduce":"","soapui": "soap",
          "maptitude": "gis", "marcosdetrabajoságil": "agile", "mathplotlib":"matplotlib", "maysal": "", "microsoftazure": "azure", "microsoftexcel": "excel", "microsoftssis": "sql server ssis", "microsoftsuite": "microsoft suite", "microsoftvisualstudio": "visual studio",
          "microsoftwindowsxp": "microsoft", "microstrategy": "micro strategy", "mlstudio": "azure machine learning studio", "msofficeyaplicacionesdesoftwarefinanciero": "microsoft office", "notspecified": "", "office365": "office", "officemysql": "office,mysql","tfs": "azure devops",
          "petenteenhojasdecálculo":"excel", ",power,":",power bi,", "powerapps": "power apps", "powerautomate": "power automate", "powerapps":"power apps", "powerpivot": "power pivot", "powerquery": "power query", "powershell": "power shell", "programaciónsas": "sas",
          "pyrosettapreferred":"pyrosetta", ",qliksense,":",qlik,", "randpython": "python", "reactnode": "react", "redshift":"aws redshift", "remedy": "", "restapi": "rest,api", "revit": "autodesk revit", ",s3,": ",aws s3,", "salesforcesoftware": "", "sasgride": "sas",
          "sqlandproceduralprogramminglanguages": "sql", "sqlserver": "sql server", "ssas": "microsoft sql server's analysis services", "ssis":"sql server integration services", "ssrs": "sql server reporting services", "superset": "apache superset", "svn":"apache subversion",
          "unixshellscripting": "unix,shell", "unixshellscripts": "unix,shell", "visualbasic": "vba", "vmwarevcp": "vmware", "waterfall": "", "webapi":"api", "workbench": "mysql workbench", "workflow":"", "wpf": "microsoft wpf", "plsql":"pl sql"}

for i in chage_P.keys():
    jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(i, chage_P[i], regex = False)
    
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(",power$", ",power bi")
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace("^power,", "power bi,")
jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace("^office,", "microsoft office,")

jobs.at[3,"Lenguajes de programación"] = 'sql,python,scala,r,sql server,oracle,mysql,no sql,redis,mongodb,azure cosmos db,apache hadoop,spark,kafka,agile,git,svn,azure devops'
jobs.at[5,"Lenguajes de programación"] = 'sql,presto,hive,python,apache airflow,luigi'
jobs.at[6,"Lenguajes de programación"] = 'python,c#,azure data factory,azure data lake,azure,azure databricks,sql'
jobs.at[8,"Lenguajes de programación"] = 'python,sql,hadoop,kafka,tensorflow,spark,pytorch,java,scala,python,r,c,c#,go,apache airflow,spark,kafka,aws'
jobs.at[9,"Lenguajes de programación"] = "oracle,sql server,teradata,unix,shell"
jobs.at[12,"Lenguajes de programación"] = 'datahub,data warehouse,tableau,google cloud platform,gquery'
jobs.at[13,"Lenguajes de programación"] = 'python,c#,.net,ansible,playbook,yaml,linux,azure,agile,azure data factory'
jobs.at[15,"Lenguajes de programación"] = 'hadoop,spark,hive,scala,pyspark,snowflake cloud data,sas,kafka,aws s3'
jobs.at[18,"Lenguajes de programación"] = "sql,azure,erwin,talend,matillion,"
jobs.at[19,"Lenguajes de programación"] = "sql,gquery,python,apache airflow"
jobs.at[20,"Lenguajes de programación"] = 'sql,python,snowflake cloud data,airflow'
jobs.at[22,"Lenguajes de programación"] = "azure data lake,python,teradata,sql"
jobs.at[23,"Lenguajes de programación"] = 'microsoft office,excel,powerpoint,word,outlook'
jobs.at[24,"Lenguajes de programación"] = 'python,spark,c,c++,sql,plotly,d3.js,matplotlib,bokeh,seaborn'
jobs.at[26,"Lenguajes de programación"] = "spark,aws sagemaker,apache airflow,databricks,kubernetes,aws,python,scala,kafka,hadoop,hive,tensorflow,pytorch,azure,sql"
jobs.at[28,"Lenguajes de programación"] = "aws,azure,google cloud platform,python,javascript,c,go,rust"
jobs.at[29,"Lenguajes de programación"] = 'python,java,c++,scala,docker,kubernetes'
jobs.at[33,"Lenguajes de programación"] = "pytorch,numpy,jax,tensorflow,keras,fastai,opencv"
jobs.at[35,"Lenguajes de programación"] = "pytorch,tensorflow,s,apache airflow,kubernetes,tfx"
jobs.at[37,"Lenguajes de programación"] = "javascript,cribl"
jobs.at[40,"Lenguajes de programación"] = "soap,sql,postgresql,sql server"
jobs.at[42,"Lenguajes de programación"] = 'azure devops,agile,go,python,java,ado,puppet,chef,ansible,azure,c,c++,java,c#'
jobs.at[46,"Lenguajes de programación"] = "python,numpy,pandas,scikit learn,tensorflow,pytorch,matplotlib,opencv,scikitimage,keras,docker"
jobs.at[48,"Lenguajes de programación"] = "r,sql,python,scala,java,c++,tableau,hadoop"
jobs.at[49,"Lenguajes de programación"] = "elastic,cloudera,apache nifi,kafka,hadoop,kusto,sql,python,java,haskell,c,c++,r"
jobs.at[50,"Lenguajes de programación"] = "r,sql,python,scala,java,c++,tableau"
jobs.at[51,"Lenguajes de programación"] = 'python,r,spark,sql,tensorflow,scikit learn,numpy,pytorch,matplotlib,shap,power bi,tableau'
jobs.at[58,"Lenguajes de programación"] = "sql,microsoft sql server's analysis services,oracle 12c,tableau"
jobs.at[61,"Lenguajes de programación"] = "sql,qlik,power bi"
jobs.at[62,"Lenguajes de programación"] = "sql"
jobs.at[65,"Lenguajes de programación"] = "excel,smartsheets,google spreadsheets"
jobs.at[66,"Lenguajes de programación"] = "grafana"
jobs.at[68,"Lenguajes de programación"] = "python,r,sql,tableau,google analytics,hive,gquery"
jobs.at[70,"Lenguajes de programación"] = "microsoft suite,powerpoint,microsoft teams,word,excel,outlook"
jobs.at[71,"Lenguajes de programación"] = "datalake,,sql,prism,snowflake cloud data"
jobs.at[72,"Lenguajes de programación"] = "sql,excel,spss,xml,javascript"
jobs.at[79,"Lenguajes de programación"] = 'snowflake cloud data,looker,sql'
jobs.at[84,"Lenguajes de programación"] = 'azure,databricks,apache spark,synapse,sql,no sql,scala,python,java,r,power shell'
jobs.at[86,"Lenguajes de programación"] = "python,java,perl,php"
jobs.at[87,"Lenguajes de programación"] = 'azure,adls,azure data factory,synapse,sql,azure iothub,azure hdinsight,azure databricks,azure cosmos db,mlstudio,azure'
jobs.at[89,"Lenguajes de programación"] = 'sql,gquery,apache airflow,luigi,airbyt,metabase,python'
jobs.at[91,"Lenguajes de programación"] = 'python,sql,snowflake cloud data,azure'
jobs.at[92,"Lenguajes de programación"] = 'snowflake cloud data,snowflake cloud data,microsoft office,sql'
jobs.at[95,"Lenguajes de programación"] = 'sql,python,apache airflow,apache,kafka,tableau'
jobs.at[96,"Lenguajes de programación"] = "python,sql,aws,lambda,aws s3,pyspark,redshift,google cloud platform"
jobs.at[98,"Lenguajes de programación"] = 'sql,data warehouse,meltano,python,aws s3,glue,athena,aws cloud watch,lambda,snowflake cloud data'
jobs.at[99,"Lenguajes de programación"] = 'excel,azure'
jobs.at[100,"Lenguajes de programación"] = "hadoop,apache nifi,angular,aws,docker,kubernetes"
jobs.at[103,"Lenguajes de programación"] = "tableau,power bi,alteryx,snowflake cloud data"
jobs.at[104,"Lenguajes de programación"] = 'microsoft office,excel,powerpoint,word,outlook'
jobs.at[105,"Lenguajes de programación"] = "excel,powerpoint"
jobs.at[106,"Lenguajes de programación"] = "php,python,node.js,mysql,oracle"
jobs.at[112,"Lenguajes de programación"] = "python,aws,azure,google cloud platform,react,iot"
jobs.at[113,"Lenguajes de programación"] = "appian,java,sql,python,agile,scrum,c++"
jobs.at[115,"Lenguajes de programación"] = 'python,r,tensorflow,gcp,aws,tableau,data studio'
jobs.at[116,"Lenguajes de programación"] = "sql server,mysql,sql cloud,azure,aws"
jobs.at[119,"Lenguajes de programación"] = "php,javascript,python,maysal,css,ajax,bootstrap,jquery,ssh,xampp"
jobs.at[121,"Lenguajes de programación"] = "git,bash,python,r,sql,pandas,numpy,scikit learn"
jobs.at[126,"Lenguajes de programación"] = "r,python,sql,oracle"
jobs.at[128,"Lenguajes de programación"] = "d3.js,ggplot,no sql,mongodb,cassandra,apache hbase,sql,hadoop,r,weka,numpy,matlab"
jobs.at[129,"Lenguajes de programación"] = "office"
jobs.at[130,"Lenguajes de programación"] = 'phyton,r,tensorflow,sql,no sql,gcp,aws,agile,tableau,data studio'
jobs.at[131,"Lenguajes de programación"] = 'postgresql,sql'
jobs.at[132,"Lenguajes de programación"] = '.net,c#,sql server,api,azure'
jobs.at[136,"Lenguajes de programación"] = 'excel,powerpoint,r,python'
jobs.at[139,"Lenguajes de programación"] = 'r,python,google sheets,google slides,excel,powerpoint'
jobs.at[140,"Lenguajes de programación"] = "sql,python,r"
jobs.at[141,"Lenguajes de programación"] = 'jira'
jobs.at[142,"Lenguajes de programación"] = 'java,javascript,sql,mysql,agile,bucket,git'
jobs.at[149,"Lenguajes de programación"] = "power bi,data warehouse,datamart"
jobs.at[150,"Lenguajes de programación"] = ".net,sql server"
jobs.at[152,"Lenguajes de programación"] = "react,node.js,soap,rest,git,svn"
jobs.at[153,"Lenguajes de programación"] = "sql server,mysql,postgresql"
jobs.at[154,"Lenguajes de programación"] = "angular,node.js,html5"
jobs.at[155,"Lenguajes de programación"] = "c#,asp.net,xml,xsl,oracle sql,sql,react,angular,jquery,html,javascript,css,mybatis,rest,api"
jobs.at[157,"Lenguajes de programación"] = "c#,sql,javascript,css,bootstrap,jquery,agile"
jobs.at[158,"Lenguajes de programación"] = "aws,sql,sql,apache,aurora,mysql"
jobs.at[159,"Lenguajes de programación"] = ""
jobs.at[164,"Lenguajes de programación"] = "sql server,mysql,sql cloud,azure,aws"
jobs.at[165,"Lenguajes de programación"] = 'node.js,vue,postgresql,redis,google cloud platform,twilio,sendgrid,rabtmq,github'
jobs.at[166,"Lenguajes de programación"] = "java,python,go,ruby,perl"
jobs.at[167,"Lenguajes de programación"] = "soc,c,c++"
jobs.at[169,"Lenguajes de programación"] = "mysql oracle,sql server,python,tableau"
jobs.at[170,"Lenguajes de programación"] = "tableau,oracle,sql server,pentaho,mysql oracle,sql server,python"
jobs.at[172,"Lenguajes de programación"] = 'vba,c#,asp.net,mvcrazor5,.net,azure,scrum,kanban'
jobs.at[176,"Lenguajes de programación"] = 'c#,asp.net,mvcrazor5,.net,java,azure,sql server,oracle,mysql,agile,scrum,kanban'
jobs.at[177,"Lenguajes de programación"] = "power bi,sql,excel"
jobs.at[178,"Lenguajes de programación"] = 'ibm infosphere'
jobs.at[180,"Lenguajes de programación"] = "sql,java,html5,css"
jobs.at[182,"Lenguajes de programación"] = "c#,asp.net,.net,java,azure,sql server,oracle,mysql,mongodb,cassandra,agile"
jobs.at[183,"Lenguajes de programación"] ="microsoft office,word,excel,powerpoint,scrum"
jobs.at[187,"Lenguajes de programación"] = '.net,c#,asp.net,css,html,javascript,bootstrap,sql,api,json,xmlajax'
jobs.at[189,"Lenguajes de programación"] = "tableau,excel"
jobs.at[190,"Lenguajes de programación"] = 'excel'
jobs.at[192,"Lenguajes de programación"] = 'excel,powerpivot,power query,power bi,sql,mysql,access'
jobs.at[194,"Lenguajes de programación"] = 'excel,sap'
jobs.at[195,"Lenguajes de programación"] = 'microsoft graph,javascript,html5,api,html,css,javascript,sql,azure,data warehouse,azure cosmos db'
jobs.at[199,"Lenguajes de programación"] = 'microsoft office,excel'
jobs.at[202,"Lenguajes de programación"] = "java,html,css,bootstrap,javascript,mysql,postgresql,sql server,sqlite"
jobs.at[203,"Lenguajes de programación"] = "java,php,sql,javascript,microsoft office,mysql,android,ios,mysql workbench,navicat,redmine,html,css"
jobs.at[205,"Lenguajes de programación"] = "microsoft"
jobs.at[211,"Lenguajes de programación"] = 'excel'
jobs.at[214,"Lenguajes de programación"] = 'excel,access,google slides,jda space planning,spaceman'
jobs.at[215,"Lenguajes de programación"] = 'excel'
jobs.at[216,"Lenguajes de programación"] = 'word,excel,powerpoint'
jobs.at[217,"Lenguajes de programación"] = "excel,vba,sql,powerpoint,microsoft visio,word,excel,access"
jobs.at[219,"Lenguajes de programación"] = "sql,excel,power bi,tableau,power bi,aws,azure,google cloud platform"
jobs.at[220,"Lenguajes de programación"] = "power bi"
jobs.at[223,"Lenguajes de programación"] = ''
jobs.at[227,"Lenguajes de programación"] = "excel,oracle sql,unix"
jobs.at[232,"Lenguajes de programación"] = 'microsoft office,excel'
jobs.at[239,"Lenguajes de programación"] = 'excel'
jobs.at[248,"Lenguajes de programación"] = "excel"
jobs.at[250,"Lenguajes de programación"] = "excel,vba,sql,postgresql,javascript"
jobs.at[251,"Lenguajes de programación"] = "spark,hadoop,cassandra,r,python,sql,no sql"
jobs.at[253,"Lenguajes de programación"] = "python,hadoop,pig,hive,scope,azure cosmos db"
jobs.at[257,"Lenguajes de programación"] = "python,sql,power bi"
jobs.at[269,"Lenguajes de programación"] = "excel"
jobs.at[270,"Lenguajes de programación"] = "excel,sql,sap,google analytics,adobe,tableau,power bi"
jobs.at[271,"Lenguajes de programación"] = "kafka,hadoop,presto,spark,kubernetes,apache airflow,graphite,beacon,sql"
jobs.at[272,"Lenguajes de programación"] = "python,spark,pyspark,sql,html,css,javascript,rest,graphql,sql,no sql,databricks,jupyter,snowflake cloud data,apache airflow,github"
jobs.at[275,"Lenguajes de programación"] = "apex,java,ajax,microsoft office"
jobs.at[280,"Lenguajes de programación"] = 'access,excel'
jobs.at[281,"Lenguajes de programación"] = "python,rstudio"
jobs.at[284,"Lenguajes de programación"] = 'mongodb,cassandra,sql,hive,r,weka,numpy,matlab'
jobs.at[285,"Lenguajes de programación"] = "aws,azure,google cloud platform,sql,excel,sql,power bi"
jobs.at[297,"Lenguajes de programación"] = "sas,sql,excel"
jobs.at[302,"Lenguajes de programación"] = "hadoop,spark,power bi,tableau,excel,sql,azure"
jobs.at[304,"Lenguajes de programación"] = 'aws,spark,hadoop,python,azure,databricks,azure blob storage,azure data factory,git'
jobs.at[306,"Lenguajes de programación"] = 'excel,spss,sas,sql,python,r,julia,vba,access'
jobs.at[309,"Lenguajes de programación"] = "spark,hadoop,cassandra,r,python,sql,no sql"
jobs.at[311,"Lenguajes de programación"] = "spark,hadoop,cassandra,r,python,sql,no sql"
jobs.at[313,"Lenguajes de programación"] = "react,python,sql,c#"
jobs.at[318,"Lenguajes de programación"] = "excel,vba,sql,postgresql,javascript"
jobs.at[326,"Lenguajes de programación"] = "power bi"
jobs.at[327,"Lenguajes de programación"] = 'sql,bash,dataiku,aws,aws emr,aws s3,aws cloud watch,vpc,spark,hive'
jobs.at[329,"Lenguajes de programación"] = "python,power bi,tableau,spss,r"
jobs.at[352,"Lenguajes de programación"] = "excel,spss,sas,sql,oracle,python,tableau,qlik,aws"
jobs.at[359,"Lenguajes de programación"] = "sql,microsoft office,word,excel,outlook,powerpoint,power bi"
jobs.at[369,"Lenguajes de programación"] = "sql,r,python,sql server management studio,oracle,power bi"
jobs.at[372,"Lenguajes de programación"] = "python,pytorch,tensorflow,argo,kubeflow,apache airflow"
jobs.at[374,"Lenguajes de programación"] = "c++,java,python,r,scala,google dialogflow,live engage,kore.ai,hadoop,spark"
jobs.at[376,"Lenguajes de programación"] = 'python,java,r'
jobs.at[378,"Lenguajes de programación"] = "javascript,sql,postgresql,react,kubernetes,docker,node.js"
jobs.at[379,"Lenguajes de programación"] = 'sql,java,unix,perl,shell,python,oracle 12c,tableau,qlikview'
jobs.at[380,"Lenguajes de programación"] = "python,aws,aws sagemaker,scikit learn,tensorflow,pytorch,kubernetes,kubeflow,docker,rest,api"
jobs.at[382,"Lenguajes de programación"] = "bash,python,matlab,r,fsl,freesurfer,spm,git,lightgbm,keras,tensorflow,pytorch,pandas,numpy"
jobs.at[390,"Lenguajes de programación"] = "sql,python,shell,google cloud platform"
jobs.at[391,"Lenguajes de programación"] = "excel,tableau,power bi"
jobs.at[393,"Lenguajes de programación"] = "gis,mapinfo,arcgis,microsoft office,adobe"
jobs.at[395,"Lenguajes de programación"] = 'excel,powerpoint'
jobs.at[396,"Lenguajes de programación"] = 'gis,word,excel,powerpoint,oracle database,oracle application express,crystal reports,arcgis'
jobs.at[399,"Lenguajes de programación"] = "python,r,javascript,sas,spss,stata,sql,excel,cognos8,dash,tableau desktop,tableau server,power bi"
jobs.at[406,"Lenguajes de programación"] = "sas,sql,microsoft office,sas,gis,tableau"
jobs.at[408,"Lenguajes de programación"] = "sql,python,r,excel,power pivot,power query,qlik,power bi,tableau,sql"
jobs.at[413,"Lenguajes de programación"] = 'sql,arcgis,excel,access,word,powerpoint,r,github'
jobs.at[414,"Lenguajes de programación"] = "matlab,python,numpy"
jobs.at[417,"Lenguajes de programación"] = 'cisco'
jobs.at[421,"Lenguajes de programación"] = "python,sql,bash,shell,spark,apache airflow,pandas,sharepoint"
jobs.at[430,"Lenguajes de programación"] = "windows,linux,microsoft office,wireshark,ios"
jobs.at[432,"Lenguajes de programación"] = "python,java,sql,javascript,aws,azure,google cloud platform,snowflake cloud data,google bigquery"
jobs.at[433,"Lenguajes de programación"] = "sql,python,git,aws,microsoft office,outlook,google suite"
jobs.at[439,"Lenguajes de programación"] = "sql,python,java,c++,scala,azure data lake,azure cosmos db,synapse,databricks"
jobs.at[440,"Lenguajes de programación"] = "sql,javascript,node.js,graphql,hasura,postgresql,react,vite,aws azure,docker"
jobs.at[441,"Lenguajes de programación"] = "sql,javascript,python,shell,oracle,google data studio"
jobs.at[443,"Lenguajes de programación"] = "sql,python,redshift,databricks,aws"
jobs.at[444,"Lenguajes de programación"] = "java,.net,python,javascript,sql,aws,azure,google cloud platform,android"
jobs.at[446,"Lenguajes de programación"] = "javascript,aws,node.js"
jobs.at[447,"Lenguajes de programación"] = "c#,.net,java,go,javascript,json,sql,sql server,azure,mongodb,azure cosmos db,json,webapi,docker"
jobs.at[450,"Lenguajes de programación"] = "python,java,r,keras,pytorch,scikit learn"
jobs.at[456,"Lenguajes de programación"] = 'autocad,revit'
jobs.at[458,"Lenguajes de programación"] = "sql,oracle,sql server"
jobs.at[462,"Lenguajes de programación"] = "sql,datalake,data warehouse,tableau"
jobs.at[463,"Lenguajes de programación"] = "sql,python,azure,power bi,excel"
jobs.at[467,"Lenguajes de programación"] = "sql,r,python,java,c#,power bi,azure,hadoop"
jobs.at[476,"Lenguajes de programación"] = "python,sql,r,amplitude,looker,tableau,power bi,sisense,superset,mixpanel"
jobs.at[480,"Lenguajes de programación"] = 'sql,python,java,hadoop,spark,ldap,vmware,docker,ansible,puppet,sql server,oracle,hive,postgresql,no sql'
jobs.at[481,"Lenguajes de programación"] = "azure,jira"
jobs.at[483,"Lenguajes de programación"] = "c++,python,sql,docker,kubernetes"
jobs.at[485,"Lenguajes de programación"] = 'c#,sql,azure data factory,azure,azure sql datawarehouse,azure data lake,azure databricks,azure cosmos db'
jobs.at[486,"Lenguajes de programación"] = "python,sql,ruby,spark,apache airflow,flask,ruby"
jobs.at[487,"Lenguajes de programación"] = ".net,c#,sql,html,css,javascript,azure,gitweb,angular,react"
jobs.at[488,"Lenguajes de programación"] = "react,redux,angular,vue,ios,android,java,.net,python,javascript,sql,aws,azure,google cloud platform"
jobs.at[490,"Lenguajes de programación"] = "sas,sql,microsoft office,gis"
jobs.at[492,"Lenguajes de programación"] = "sql,oracle,sql server,sql management studio,excel"
jobs.at[494,"Lenguajes de programación"] = "sql,gis,word,excel,powerpoint,microsoft visio,oracle database,apex,crystal reports,arcgis,bpmn"
jobs.at[497,"Lenguajes de programación"] = "python,r,javascript,sql,microsoft sql server's analysis services,sql server reporting services,r,excel,cognos8,spss,dash,tableau desktop,tableau server,power bi"

jobs["Lenguajes de programación"] = jobs["Lenguajes de programación"].str.replace(",,", ",")

In [20]:
all_Planguages = []
for i in jobs["Lenguajes de programación"].str.split(","):
    if isinstance(i, list):
        all_Planguages = [*all_Planguages, *i]
unique_Planguagues = list(set(all_Planguages))
unique_Planguagues.sort()
unique_Planguagues.remove('')
unique_Planguagues

['.net',
 'access',
 'adls',
 'ado',
 'adobe',
 'agile',
 'airbyt',
 'airflow',
 'ajax',
 'alation',
 'alteryx',
 'amplitude',
 'android',
 'angular',
 'ansible',
 'apache',
 'apache airflow',
 'apache hadoop',
 'apache hbase',
 'apache nifi',
 'apache spark',
 'apex',
 'api',
 'appian',
 'arcgis',
 'argo',
 'asp.net',
 'athena',
 'aurora',
 'autocad',
 'aws',
 'aws azure',
 'aws cloud watch',
 'aws ec2',
 'aws emr',
 'aws glue',
 'aws redshift',
 'aws s3',
 'aws sagemaker',
 'azure',
 'azure blob storage',
 'azure cosmos db',
 'azure data factory',
 'azure data lake',
 'azure databricks',
 'azure devops',
 'azure hdinsight',
 'azure iothub',
 'azure sql datawarehouse',
 'bamboo',
 'bash',
 'beacon',
 'bokeh',
 'bootstrap',
 'bpmn',
 'bucket',
 'c',
 'c#',
 'c++',
 'cassandra',
 'chef',
 'circleci',
 'cisco',
 'cloud',
 'cloudera',
 'cognos8',
 'cribl',
 'crystal reports',
 'css',
 'cypress',
 'd3.js',
 'dash',
 'data studio',
 'data warehouse',
 'databricks',
 'datahub',
 'dataiku',
 

¿Cuáles son los lenguajes de programación/software más demandados?

In [21]:
count_software = pd.Series([i for i in all_Planguages if i is not '']).value_counts()

Vamos a obtener aquellos que aparezcan al menos en un 5% de la muestra

In [22]:
count_software[count_software>=jobs.shape[0]*.05]

sql           224
python        174
excel         150
r              82
power bi       69
tableau        59
java           55
aws            43
azure          41
javascript     39
sas            28
spark          27
hadoop         26
dtype: int64

In [23]:
for i in count_software.items():
    print(i)

('sql', 224)
('python', 174)
('excel', 150)
('r', 82)
('power bi', 69)
('tableau', 59)
('java', 55)
('aws', 43)
('azure', 41)
('javascript', 39)
('sas', 28)
('spark', 27)
('hadoop', 26)
('oracle', 25)
('mysql', 24)
('c#', 24)
('sql server', 24)
('c++', 23)
('microsoft office', 22)
('powerpoint', 21)
('word', 19)
('spss', 17)
('scala', 17)
('docker', 16)
('.net', 15)
('postgresql', 15)
('pytorch', 14)
('tensorflow', 14)
('vba', 13)
('google cloud platform', 13)
('css', 13)
('agile', 13)
('kubernetes', 12)
('apache airflow', 12)
('office', 12)
('git', 12)
('react', 11)
('html', 11)
('ruby', 11)
('snowflake cloud data', 11)
('mongodb', 10)
('kafka', 10)
('gis', 10)
('hive', 10)
('sap', 9)
('node.js', 9)
('no sql', 9)
('c', 9)
('numpy', 9)
('qlik', 8)
('github', 8)
('shell', 8)
('outlook', 8)
('arcgis', 7)
('scrum', 7)
('go', 7)
('stata', 7)
('json', 7)
('matlab', 7)
('api', 7)
('scikit learn', 7)
('jquery', 7)
('azure cosmos db', 7)
('xml', 7)
('access', 7)
('pandas', 7)
('cassandra', 7)


Lo anterior es por lenguajes o software en específico, vamos a sacarlo por netos

In [24]:
netos = pd.DataFrame([i for i in all_Planguages if i is not ''], columns=["Software"])

In [25]:
aux_dict = {"^apache.+":"apache",
            "^aws.+": "aws",
            "^azure.+":"azure", 
            "excel":"excel/vba", "vba":"excel/vba",
            "^c$":"c/c#/c++", "^c#$":"c/c#/c++", "^c\+\+$":"c/c#/c++",
            "ggplot":"r", "rstudio":"r", 
            "git.*": "git", 
            "^google.+": "google", 
            ".*sql.*":"sql",
            "jupyter": "python", "matplotlib":"python", "tensorflow":"python", "scikit learn": "python", "scikitimage": "python", "seaborn": "python", "numpy": "python", "pandas": "python","^py\w+": "python",
            "oracle": "oracle", 
            "microsoft": "microsoft/office", "powerpoint":"microsoft/office", "word":"microsoft/office",
            "scrum":"scrum/agile", "agile":"scrum/agile", "^tableau.+":"tableau"}

for i in aux_dict.keys():
    netos["Software"] = netos["Software"].map(lambda x: aux_dict[i] if bool(re.search(i,x)) else x)

In [26]:
netos["Software"].value_counts()

sql                 319
python              242
excel/vba           163
r                    85
microsoft/office     71
                   ... 
maysal                1
iot                   1
playbook              1
hex                   1
msstack               1
Name: Software, Length: 236, dtype: int64

In [27]:
count_netos = netos["Software"].value_counts()
count_netos[count_netos>=jobs.shape[0]*.05]

sql                 319
python              242
excel/vba           163
r                    85
microsoft/office     71
power bi             69
azure                66
tableau              63
aws                  62
c/c#/c++             56
java                 55
javascript           39
oracle               30
google               29
sas                  28
spark                27
hadoop               26
Name: Software, dtype: int64

In [28]:
for i in count_netos.items():
    print(i)

('sql', 319)
('python', 242)
('excel/vba', 163)
('r', 85)
('microsoft/office', 71)
('power bi', 69)
('azure', 66)
('tableau', 63)
('aws', 62)
('c/c#/c++', 56)
('java', 55)
('javascript', 39)
('oracle', 30)
('google', 29)
('sas', 28)
('spark', 27)
('hadoop', 26)
('apache', 22)
('git', 22)
('scrum/agile', 20)
('scala', 17)
('spss', 17)
('docker', 16)
('.net', 15)
('css', 13)
('office', 12)
('kubernetes', 12)
('html', 11)
('ruby', 11)
('react', 11)
('snowflake cloud data', 11)
('kafka', 10)
('hive', 10)
('mongodb', 10)
('gis', 10)
('sap', 9)
('node.js', 9)
('outlook', 8)
('shell', 8)
('qlik', 8)
('arcgis', 7)
('matlab', 7)
('xml', 7)
('access', 7)
('cassandra', 7)
('json', 7)
('stata', 7)
('jquery', 7)
('api', 7)
('unix', 7)
('go', 7)
('databricks', 6)
('rest', 6)
('adobe', 6)
('angular', 6)
('gquery', 6)
('php', 6)
('data warehouse', 6)
('data studio', 5)
('ansible', 5)
('keras', 5)
('asp.net', 5)
('bootstrap', 5)
('bash', 4)
('html5', 4)
('kanban', 4)
('looker', 4)
('power query', 4)
('

Con lo anterior, hacemos una variable que determine cuantos lenguajes/habilidades de programación se están solicitando y una variables indicadoras para los lenguajes/habilidades de programación importantes o más solicitados

In [29]:
def empy_len(lista):
    if isinstance(lista, list):
        return(len(lista))
    else:
        return 0
jobs["Cantidad de lenguajes requeridos"] = jobs["Lenguajes de programación"].str.split(",").map(empy_len)

In [30]:
significant_netos = count_netos[count_netos>=jobs.shape[0]*.05]

In [31]:
significant_netos

sql                 319
python              242
excel/vba           163
r                    85
microsoft/office     71
power bi             69
azure                66
tableau              63
aws                  62
c/c#/c++             56
java                 55
javascript           39
oracle               30
google               29
sas                  28
spark                27
hadoop               26
Name: Software, dtype: int64

In [32]:
for i in aux_dict.keys():
    netos["Software"] = netos["Software"].map(lambda x: aux_dict[i] if bool(re.search(i,x)) else x)

In [33]:
def search_in_list(pattern, lista):
    for i in lista:
        if bool(re.search(pattern,i)):
            return True
    return False

def check_string(something):
    if isinstance(something, str):
        if len(something)>0:
            return something
        else: return " "
    else:
        return " "

def dummy_lenguageProgramming(data_original):
    data = data_original.copy()
    aux_dict = {".*sql.*":"sql",
            "^jupyter$": "python", "^matplotlib$":"python", "^tensorflow$":"python", "^scikit learn$": "python", "^scikitimage$": "python", "^seaborn$": "python", "^numpy$": "python", "^pandas$": "python","^py\w+": "python",
            "^excel$":"excel/vba", "^vba$":"excel/vba",
            "^ggplot$":"r", "^rstudio$":"r", 
            "^microsoft$": "microsoft/office", "^powerpoint$":"microsoft/office", "^word$":"microsoft/office",
            "^power bi$":"power bi",
            "^azure.+":"azure",
            "^apache.+":"apache",
            "^tableau.+":"tableau",
            "^aws.+": "aws",
            "^c$":"c/c#/c++", "^c#$":"c/c#/c++", "^c\+\+$":"c/c#/c++",
            "^java$":"java",
            "^javascript$":"javascript",
            "^oracle$": "oracle", 
            "^google.+": "google", 
            "^sas$": "sas",
            "^spark$": "spark",
            "^hadoop$": "hadoop"}

    #Creamos las variables
    for i in list(set(aux_dict.values())):
        data[i] = 0

    #Ahora, hacemos las identificaciones:
    for i in aux_dict.keys():
        data[aux_dict[i]] = data[aux_dict[i]] + data["Lenguajes de programación"].map(lambda x: search_in_list(i, check_string(x).split(",")))

    #Arreglamos cuando hubieron duplicados:
#    for i in list(set(aux_dict.values())):
#        data[i] = data[i].map(lambda x: 1 if x > 1 else x)
    
    return data

Veamos como funciona la función anterior:

In [34]:
dummy_lenguageProgramming(jobs).head(3)

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,Modalidad,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio,Lenguajes de programación,Modalidad.1,Estudios previos,Experiencia (Años),Habilidades blandas,Idiomas,c/c#/c++,sql,oracle,tableau,hadoop,javascript,apache,aws,spark,sas,r,microsoft/office,python,power bi,azure,java,excel/vba,google
0,Data Engineer with Azure Synapse,4urHires INC,not specified,not specified,8800.0,9600.000000,USD,Azure EngineerStrong experience in SynapseJob ...,not specified,2022-09-16,2022-10-16,OTHER,EUA,hour,2,Not specified,0,9200.000000,"azure,synapse","Remote,","Azure engeneer,","Data engineer: 10 years, azure synapse: 5 years",-,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Data Engineer,MARS Group,not specified,not specified,6348.8,9600.000000,USD,Description: Data is a critical driver of this...,not specified,2022-09-14,2022-10-16,FULL_TIME,EUA,hour,5,Not specified,1,7974.400000,"java,scala,aws,docker,kubernetes","Remote,","B.S. degree in Computer Science or related field,","2+ years of experience in data engineering, j...","Ability to adapt and learn new technologies, ...",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Data Engineer - ENTRY LEVEL,PCS Global Tech,California,Los Angeles,5000.0,5833.333333,USD,Responsibilities· Analyze and organize raw dat...,Tecnologías de la información,2022-09-15,2022-10-16,FULL_TIME,EUA,year,3,Junior,1,5416.666667,"java,python,sql","Multiple locations,","Degree in Computer Science, IT, or similar fie...","Trainee,","Collaborate,",NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


Ahora vamos a trabajar con la variable de idioma extra

In [35]:
jobs["Idiomas"] = jobs["Idiomas"].fillna("not specified")
jobs["Idiomas"] = jobs["Idiomas"].str.replace("-", "not specified")

In [36]:
chage_L = {"English":"Inglés",",comunicación verbal y escrita en español e inglés,":"Bilingüe",
           "not specifiednot specified":"not specified","able to communicate fluently in English both verbally and in writing, ":"Inglés avanzado/fluido",
           "english":"Inglés","Inglés avanzado y fluido, ":"Inglés avanzado/fluido",
           "\.":"","Bilingue":"Bilingüe","comunicación verbal y escrita en español e inglés":"Bilingüe",
           "Inglés, ":"Inglés","Advanced English, ":"Inglés avanzado/fluido"," ingles 90% avanzado, ":"Inglés avanzado/fluido",
           "Inglés Intermedio ":"Inglés Intermedio","Inglés conversacional ":"Inglés Intermedio",".Inglés intermedio conversacional, ":"Inglés Intermedio",
           "Ingles fluido,":"Inglés avanzado/fluido","Nivel avanzado del idioma inglés, ":"Inglés avanzado/fluido","BILINGÜE, Inglés y español fluidos \(escrito y oral\), ":"Bilingüe",
           "English, inglés intermedio, ":"Inglés Intermedio","bilingüe, ingles avanzado, inglés avanzado, ":"Inglés avanzado/fluido","ingles":"Inglés",
           "ingles intermedio avanzado, ":"Inglés intermedio/avanzado","inglés conversacional B1 o B2, ":"Inglés intermedio"," ingles conversacional avanzado B1 o B2":"Inglés intermedio",
           "ingles avanzado conversacional, ":"Inglés avanzado/fluido","ingles avanzado, ":"Inglés avanzado/fluido",
           "Inglés:  Intermedionot specifiedAvanzadoHorario":"Inglés intermedio","DashboardsInglés básico o conversacional, ":"Inglés básico",
           "inglés,":"Inglés","Inglés ":"Inglés","CloudInglés AvanzadoSueldo, ":"Inglés avanzado/fluido",
           ".Bilingüe, ":"Bilingüe","Idioma:Ingles 80% , ":"Inglés avanzado/fluido",
           "ingles nivel  intermedio, ":"Inglés intermedio"," bilingüe":"Bilingüe",
           "Inglés intermedio a avanzado, ":"Inglés intermedio/avanzado","MóvilIngles intermedionot specifiedavanzadoDisponibilidad,":"Inglés intermedio",
           " Inglés conversacional,":"Inglés básico","bilingüe":"Bilingüe","Inglés intermedio a avanzado,":"Inglés intermedio/avanzado",
           "Tecnico intermedio ,hablado, escrito, leído,":"Inglés intermedio","Inglés Intermedio \(comprobable\),":"Inglés intermedio",
           "Inglés \(leído\),":"Inglés básico","Inglés básico":"Inglés básico","Dominio del idioma inglés \(Intermedio B2 o equivalente, ":"Inglés intermedio",
           "inglés conversacional, B1 o B2,: Debe ser capaz de comunicarse de forma básica durante situaciones ":"Inglés intermedio","Inglés básicoResponsabilidades,":"Inglés básico",
           "Ingles Intermedio/Avanzado":"Inglés intermedio/avanzado","Inglés:  Intermedio / Avanzado":"Inglés intermedio/avanzado",
           "Ingles":"Inglés","inglés avanzado":"Inglés avanzado","Inglés: Intermedionot specifiedavanzado":"Inglés intermedio",
           "Ingles \(deseable\)":"Inglés","Ingles intermedio":"Inglés intermedio",
           "Inglés conversacional indispensable":"Inglés intermedio","Inglés  intermedio o Avanzando":"Inglés intermedio/avanzado",
           "Inglés avanzado y fluido":"Inglés avanzado/fluido","Conversational English":"Inglés intermedio",
           "English/Spanish\), Proficiency English":"Inglés avanzado/fluido","Inglés intermedio o Avanzando":"Inglés avanzado/fluido",
           "English advance":"Inglés avanzado/fluido","English/Spanish":"Bilingüe",
           "not especified":"not specified","Ingles avanzado":"Inglés avanzado/fluido",
           "Ingles conversacional":"Inglés intermedio","Inglés Intermedio / Avanzado.":"Inglés avanzado/fluido",
           "Inglés intermedio":"Inglés intermedio","Ingles nivel intermedio":"Inglés intermedio",
           "Inglés Intermedio":"Inglés intermedio","Ingles intermedio avanzado":"Inglés intermedio/avanzado",
           "Flexible Language ":"Bilingüe","Bilingual ":"Bilingüe","English essential, French essential ":"Inglés, French",
           "english, Japanese":"Inglés, Japanese","English, German":"Inglés, German",
           "English  spelling, punctuation, grammar and arithmetic":"Inglés avanzado/fluido",
           "English, French":"Inglés, French","English, Spanish":"Bilingüe"}

for i in chage_L.keys():
    jobs["Idiomas"] = jobs["Idiomas"].str.replace("^" + i + "$", chage_L[i], regex = True)
    
jobs.at[186, "Idiomas"] = "Inglés intermedio"
jobs.at[187, "Idiomas"] = "Inglés básico"
jobs["Idiomas"] = jobs["Idiomas"].str.replace("^$", "not specified", regex = True)

Ya tenemos bien codificada la variable

In [37]:
for i in jobs["Idiomas"].unique():
    print(i)

not specified
Inglés
Bilingüe
Inglés avanzado/fluido
Inglés intermedio
Inglés intermedio/avanzado
Inglés básico
Inglés avanzado
Inglés, French
Inglés, Japanese
Inglés, German


Vamos con la Modalidad.1

In [38]:
jobs["Modalidad.1"] = jobs["Modalidad.1"].fillna("not specified")
jobs["Modalidad.1"] = jobs["Modalidad.1"].str.replace("^-$", "not specified")
jobs["Modalidad.1"] = jobs["Modalidad.1"].str.lower()

In [39]:
change_Mod = {
    "remote,":"remoto","multiple locations, ":"híbrido","hybrid":"híbrido",
"one location":"presencial"," remote,  in-office ":"híbrido","remote":"remoto",
"remotely , one of the following locations, remote,":"remoto","#li-remote":"remoto",
"#li-remote,":"remoto","remote, location":"remoto"," remote-first , hybrid, ":"remoto/híbrido",
"location,":"presencial","location, ":"presencial","located, ":"presencial",
"remote-first, hybrid,":"remoto/híbrido","one location, ":"presencial","on the road, ":"remoto",
"on the road":"remoto","remote, ":"remoto","remote ,":"remoto","at home, ":"remoto",
"presencial,":"presencial","one location  ":"presencial","hybrid/remote,":"remoto/híbrido",
"remotely":"remoto","on-site, ":"presencial"," one location":"presencial","multiple locations":"híbrido",
"remoto, ":"remoto","one location.  ":"presencial"," remote":"remoto"," hybrid, ":"híbrido",
"one location, remote, ":"remoto","hybrid, remote , onsite ":"híbrido","one location,":"presencial",
"remotely, remote, ":"remoto","remote ":"remoto","trabajar desde casa,":"remoto",
" remote , ":"remoto","no remoto, ":"presencial","trabajar desde casa, ":"remoto",
"hibrida,":"híbrido"," home office,":"remoto","híbrido,":"híbrido","home office temporalmente, ":"remoto/híbrido",
"remoto":"remoto","oficina, ":"presencial","híbridojob":"híbrido","home officetipo, ":"remoto",
"from homeget , ":"remoto","home office 3 días y 2 en oficina, ":"híbrido","hibrido":"híbrido",
"home office inicialmente":"remoto/híbrido","temporalmente híbrido, después presencial, ":"remoto/híbrido",
"descuentotrabajar desde casatipos, ":"remoto","remota":"remoto","hibridoobjetivo, trabajar desde casa, ":"híbrido",
"000hibrido, ":"híbrido","home office":"remoto","home office, ":"remoto","híbridoü ,":"híbrido",
"presencial, ":"presencial",".":"not specified","presencial ":"presencial","home officehabilidadesmanejode, ":"remoto",
"híbrida,":"híbrido","en sitio, ":"presencial","home office,":"híbrido","presencial.":"presencial",
"home office ,  oficina,":"híbrido","oficina,  home office, ":"híbrido","trabajar desde casavales , ":"remoto",
"comitán o home officeescolaridad, ":"híbrido","remoto, en sitio, ":"híbrido",".-":"not specified",
"tecnologías de la información y sistemasidiomas,":"not specified","tiempo completo":"not specified",
"híbrido":"híbrido","presencial":"presencial","híbrido ":"híbrido","hibrido, remote":"híbrido",
"hybrid ":"híbrido","flexible":"híbrido"
}

for i in change_Mod.keys():
    jobs["Modalidad.1"] = jobs["Modalidad.1"].str.replace("^" + i + "$", change_Mod[i], regex = True)
    
jobs["Modalidad.1"] = jobs["Modalidad.1"].str.replace("^$", "not specified", regex = True)

Podemos sacar una indicadora de si es licenciatura, maestría, doctorado, pasante. Para esto vamos a usar solo expresiones regulares y también sobre áreas: estadística, datos, finanzas, economía, negocios, matemáticas, computación, física, ingeniería, biology, actuaría

In [40]:
jobs["Estudios previos"] = jobs["Estudios previos"].fillna("not specified")
jobs["Estudios previos"] = jobs["Estudios previos"].str.replace("^-$", "not specified")
jobs["Estudios previos"] = jobs["Estudios previos"].str.lower()

In [41]:
def dummy_school(data_original):
    data = data_original.copy()
    relevant_carrers = {"licenciatura": "licenciatura", "lic\.": "licenciatura", "bachelor":"licenciatura",
                       "maest": "maestría", "master": "maestría", "docto":"doctorado/postdoctorado", 
                       "trun": "carrera trunca/bachillerato", "bachill": "carrera trunca/bachillerato"}
    scholar_grade = {"estad": "estadística", "stati": "estadística", 
                     "datos":"ciencia/ingienería de datos", "data": "ciencia/ingienería de datos",
                    "finanz": "finanzas", "financ": "finanzas","econo": "economía", 
                    "negocios":"business", "business": "business",
                    "mate": "matemáticas", "math":"matemáticas","comput": "computación",
                    "física": "física", "phy": "física", "inge": "ingeniería", "engi": "ingeniería",
                    "bio": "biology/bioinformatics/etc.",
                    "actua": "actuaría"}

#Creamos las variables
    for i in list(set(relevant_carrers.values())):
        data[i] = 0
    for i in list(set(scholar_grade.values())):
        data[i] = 0
        
    #Ahora, hacemos las identificaciones:
    for i in relevant_carrers.keys():
        data[relevant_carrers[i]] = data[relevant_carrers[i]] + data["Estudios previos"].map(lambda x: bool(re.search(i,check_string(x))))
    for i in scholar_grade.keys():
        data[scholar_grade[i]] = data[scholar_grade[i]] + data["Estudios previos"].map(lambda x: bool(re.search(i,check_string(x))))

    #Arreglamos cuando hubieron duplicados:
#    for i in list(set(relevant_carrers.values())):
#        data[i] = data[i].map(lambda x: 1 if x > 1 else x)
#    for i in list(set(scholar_grade.values())):
#        data[i] = data[i].map(lambda x: 1 if x > 1 else x)

    return data

In [42]:
dummy_school(jobs).head(1)

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,Modalidad,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio,Lenguajes de programación,Modalidad.1,Estudios previos,Experiencia (Años),Habilidades blandas,Idiomas,licenciatura,carrera trunca/bachillerato,doctorado/postdoctorado,maestría,business,computación,biology/bioinformatics/etc.,economía,finanzas,matemáticas,ingeniería,actuaría,ciencia/ingienería de datos,física,estadística
0,Data Engineer with Azure Synapse,4urHires INC,not specified,not specified,8800.0,9600.0,USD,Azure EngineerStrong experience in SynapseJob ...,not specified,2022-09-16,2022-10-16,OTHER,EUA,hour,2,Not specified,0,9200.0,"azure,synapse",remoto,"azure engeneer,","Data engineer: 10 years, azure synapse: 5 years",-,not specified,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Tenemos que arreglar la variable Requerimiento educación

In [43]:
jobs["Requerimiento educación"] = jobs["Estudios previos"].map(lambda x: 0 if x == "not specified" else 1)

Habilidades blandas

In [44]:
jobs["Habilidades blandas"] = jobs["Habilidades blandas"].fillna("not specified")
jobs["Habilidades blandas"] = jobs["Habilidades blandas"].str.replace("^-$", "not specified")
jobs["Habilidades blandas"] = jobs["Habilidades blandas"].str.lower()

Lo demás, lo haremos mediante procesamiento de lenguaje natural

Años de experiencia

Wen modificó/mejoró la variable de descripción, así que vamos a tomarla y quitar algunos registros que se descartan también

In [45]:
jobs.drop([127, 173, 175, 179, 185, 197], inplace=True)
jobs.reset_index(drop=True, inplace=True)

In [46]:
jobs_wen_NV_v2 = pd.read_csv("LastClean/DataScience_jobs_wen_NewVariables_V2.csv")

In [47]:
for i in range(jobs_wen_NV_v2.shape[0]):
    jobs.at[i, "Descripción"] = jobs_wen_NV_v2.at[i, "Descripción.1"]
    jobs.at[i, "Experiencia (Años) "] = jobs_wen_NV_v2.at[i, "Experiencia (Años) "]

Y ahora obtenemos los años de experiencia

In [48]:
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].fillna("not specified")
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.replace("^-+$", "not specified")
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.replace("^\.-$", "not specified")
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.lower()

In [49]:
change_year = {"recién egresado": "0", "sin experiencia": "0", "6 meses": "0.5"}

for i in change_year.keys():
    jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.replace(i, change_year[i])
    
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.replace("^$", "not specified")

In [50]:
def there_is_number(lista):
    if len(lista)>=1:
        return max(lista)
    else:
        return np.nan
    
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].str.findall("\d+").map(lambda x: there_is_number(x))
jobs["Experiencia (Años) "] = jobs["Experiencia (Años) "].astype('float')

Con esto, podemos identificar si un empleo solicita años de experiencia o no

In [51]:
def not_experience(data_original):
    data = data_original.copy()
    data["not_experience"] = data["Experiencia (Años) "].map(lambda x: 1 if np.isnan(x) else 0)
    return data

In [57]:
not_experience(jobs)

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,País,Esquema de pago,Cantidad de lenguajes requeridos,Nivel de experiencia,Requerimiento educación,Salario promedio,Lenguajes de programación,Modalidad.1,Estudios previos,Experiencia (Años),Habilidades blandas,Idiomas,not_experience
0,Data Engineer with Azure Synapse,4urHires INC,not specified,not specified,8800.000000,9600.000000,USD,Azure Engineer Strong experience in SynapseJob...,not specified,2022-09-16,2022-10-16,EUA,hour,2,Not specified,1,9200.000000,"azure,synapse",remoto,"azure engeneer,",10.0,not specified,not specified,0
1,Data Engineer,MARS Group,not specified,not specified,6348.800000,9600.000000,USD,Description: Data is a critical driver of this...,not specified,2022-09-14,2022-10-16,EUA,hour,5,Not specified,1,7974.400000,"java,scala,aws,docker,kubernetes",remoto,"b.s. degree in computer science or related field,",2.0,"ability to adapt and learn new technologies, ...",not specified,0
2,Data Engineer - ENTRY LEVEL,PCS Global Tech,California,Los Angeles,5000.000000,5833.333333,USD,Responsibilities· Analyze and organize raw dat...,Tecnologías de la información,2022-09-15,2022-10-16,EUA,year,3,Junior,1,5416.666667,"java,python,sql",híbrido,"degree in computer science, it, or similar fie...",0.0,"collaborate,",not specified,0
3,Entry-Level Azure Data Engineer,"Antra, Inc",not specified,not specified,5166.666667,5833.333333,USD,"About usAntra, Inc is a leading IT consultancy...",Tecnologías de la información,2022-09-13,2022-10-16,EUA,year,18,Junior,1,5500.000000,"sql,python,scala,r,sql server,oracle,mysql,no ...",remoto,"minimum bachelor’s degree in data science, bus...",0.0,"excellent problem-solving skills, ability to ...",not specified,0
4,Data Engineer only for OPT EAD,CloudBig Technology,not specified,not specified,8000.000000,9600.000000,USD,Work with Technical and business team to under...,not specified,2022-09-15,2022-10-16,EUA,hour,2,Not specified,1,8800.000000,"sql,data warehouse",remoto,master's,11.0,"work with other,",not specified,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Sr. Business Systems Analyst,Infotek Consulting Services,Ontario,Toronto,6000.000000,8400.000000,CAD,"Need to know all: Confluence, Jira, Excel, Tab...",Tecnologías de la información,2022-09-16,2022-10-16,CAN,hour,7,Senior,0,7200.000000,"sql,python,json,qtest,zephyr,hp alm,hp qc",not specified,not specified,5.0,"excellent communication, negotiation and stake...",not specified,0
502,Sr. Business Analyst,The Getch,Ontario,Toronto,9600.000000,10800.000000,CAD,The Opportunity: Business/Systems Infr/Cloud A...,Tecnologías de la información,2022-08-30,2022-10-16,CAN,hour,2,Senior,1,10200.000000,"aws,",híbrido,b.s. degree in finance or equivalent,8.0,not specified,not specified,0
503,Sr. Business Analyst Capital Market,TechAheadCorp,Ontario,Toronto,6250.000000,NaN,CAD,- Minimum 8+ years of direct working experienc...,not specified,2022-09-09,2022-10-16,CAN,year,1,Senior,0,6250.000000,,not specified,not specified,8.0,not specified,not specified,0
504,Sr. Analyst BI,FinneyTaylor Consulting Group Ltd.,Alberta,Edmonton,4374.755000,5625.000000,CAD,Job Description : Our Client is looking for a ...,not specified,2022-09-02,2022-10-16,CAN,year,5,Senior,1,4999.877500,"sql,microsoft sql server's analysis services,d...",not specified,"bachelor’s degree in a related field (e.g., co...",5.0,"can work independently, strong oral and writte...",not specified,0


Cambiamos nombres de las columnas y está lista para ser analizada

In [56]:
jobs.drop("Modalidad", axis = 1, inplace=True)

In [61]:
jobs.rename(columns={"Nivel de experiencia": "Título ofrecido", 
                     "Lenguajes de programación": "Lenguajes de programación/tecnologías requeridas",
                    "Modalidad.1": "Modalidad"}, inplace=True)

In [63]:
jobs.head(1)

,Puesto laboral,Empresa,Estado,Ciudad,Salario mínimo,Salario máximo,Tipo de moneda,Descripción,Giro de la empresa,Fecha de publicación,Vigencia de la oferta,País,Esquema de pago,Cantidad de lenguajes requeridos,Título ofrecido,Requerimiento educación,Salario promedio,Lenguajes de programación/tecnologías requeridas,Modalidad,Estudios previos,Experiencia (Años),Habilidades blandas,Idiomas
0,Data Engineer with Azure Synapse,4urHires INC,not specified,not specified,8800.0,9600.0,USD,Azure Engineer Strong experience in SynapseJob...,not specified,2022-09-16,2022-10-16,EUA,hour,2,Not specified,1,9200.0,"azure,synapse",remoto,"azure engeneer,",10.0,not specified,not specified


In [64]:
jobs.to_csv("final_jobs_dataScience.csv", index=False)